In [1]:
merges = {(101, 102): 300, (103, 104): 301}        # example merges
special_tokens = {"<|endoftext|>": 1000}

vocab = {}
vocab.update({idx: bytes([idx]) for idx in range(256)})
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
for special, idx in special_tokens.items():
    vocab[idx] = special.encode("utf-8")

In [2]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [3]:
res = "hello"

res.encode("utf-8")

b'hello'

In [4]:
def get_stats(ids, counts=None):
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids)-1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [5]:
# ====== DEBUG TRAIN WITHOUT CLASS ======
text = "abcdefgabcd"
text_bytes = text.encode("utf-8")
ids = list(text_bytes)
print("text_bytes:",text_bytes)
print("Initial ids:", ids)

text_bytes: b'abcdefgabcd'
Initial ids: [97, 98, 99, 100, 101, 102, 103, 97, 98, 99, 100]


In [6]:
vocab = {i: bytes([i]) for i in range(256)}
vocab # {0: b'\x00',
      #  1: b'\x01',
      #  2: b'\x02',
      #  3: b'\x03',
      #  4: b'\x04',
      #  5: b'\x05',
      # ...........}

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

verbose here is a switch.
It means: “Should the function print extra information?”

Beginner-level explanation:

verbose=False
→ function is quiet
→ no prints
→ only does training

verbose=True
→ function prints every merge step
→ shows what pair was merged
→ shows new token id
→ shows how many times the pair appeared

Think of it like “debug mode ON/OFF”.


Example:

If you call:

train(text="aaaaab", vocab_size=260, verbose=True)


You will see prints like:
merge 1/4: (97, 97) -> 256 (b'aa') had 2 occurrences
merge 2/4: (256, 97) -> 257 (b'aaa') had 1 occurrences
...


If you call:
train(text="aaaaab", vocab_size=260, verbose=False)
You see nothing printed.


So:

verbose = True → show internal training details
verbose = False → stay silent

In [7]:
# ====== DEBUG TRAIN WITHOUT CLASS ======
text_bytes = text.encode("utf-8")         # text_bytes: b'abcdefgabcd'
ids = list(text_bytes)                    # ids: [97, 98, 99, 100, 101, 102, 103, 97, 98, 99, 100]

print("Initial ids:", ids)

vocab = {i: bytes([i]) for i in range(256)}   # {0: b'\x00', 1: b'\x01', 2: b'\x02', ........ }
merges = {}

vocab_size = 260       # very small vocabulary
num_merges = vocab_size - 256           # num_merges = 4

for i in range(num_merges):             # num_merges = 4
    stats = get_stats(ids)              # Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    print(f"\nStep {i+1} stats:", stats)
 
    pair = max(stats, key=stats.get)    # (97, 98)
    print("Most common pair:", pair) 

    idx = 256 + i                       # idx = 257,258,259,260 ...
    print("New token index:", idx)

    ids = merge(ids, pair, idx)         # [256, 99, 100, 101, 102, 103, 256, 99, 100]
    print("New ids:", ids)

    merges[pair] = idx                  # merges eng ko'p uchragan juftliklarni saqlab boradi va agar shu dict ichidagi juftlik kelsa 2 ta son bittaga o'tadi misol uchun (97,98)--> 256
    print(f"merges: {merges}")        
    vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
    # vocab[idx] = 257, --> vocab[pair[0]] = 97, --> vocab[pair[1]] = 98
    # vocab = 257 = vocab[97] + vocab[98]

Initial ids: [97, 98, 99, 100, 101, 102, 103, 97, 98, 99, 100]

Step 1 stats: {(97, 98): 2, (98, 99): 2, (99, 100): 2, (100, 101): 1, (101, 102): 1, (102, 103): 1, (103, 97): 1}
Most common pair: (97, 98)
New token index: 256
New ids: [256, 99, 100, 101, 102, 103, 256, 99, 100]
merges: {(97, 98): 256}

Step 2 stats: {(256, 99): 2, (99, 100): 2, (100, 101): 1, (101, 102): 1, (102, 103): 1, (103, 256): 1}
Most common pair: (256, 99)
New token index: 257
New ids: [257, 100, 101, 102, 103, 257, 100]
merges: {(97, 98): 256, (256, 99): 257}

Step 3 stats: {(257, 100): 2, (100, 101): 1, (101, 102): 1, (102, 103): 1, (103, 257): 1}
Most common pair: (257, 100)
New token index: 258
New ids: [258, 101, 102, 103, 258]
merges: {(97, 98): 256, (256, 99): 257, (257, 100): 258}

Step 4 stats: {(258, 101): 1, (101, 102): 1, (102, 103): 1, (103, 258): 1}
Most common pair: (258, 101)
New token index: 259
New ids: [259, 102, 103, 258]
merges: {(97, 98): 256, (256, 99): 257, (257, 100): 258, (258, 101): 2